In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn
import matplotlib as pyplot
from ggplot import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# suppress scientific notation
pd.options.display.float_format = '{:.2f}'.format

# Input data files are available in the "../input/" directory.
dfTrainRaw = pd.read_csv("./input/train.csv", low_memory = False)

# List all files under the input directory
import os
for dirname, _, filenames in os.walk('./kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
# Inspect data.frame
dfTrainRaw.head(5)

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.00,77.00,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.00,77.00,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.00,77.00,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.00,77.00,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.00,77.00,8,SW


In [3]:
# unique values for DefendersInTheBox
xx = dfTrainRaw.DefendersInTheBox.unique()
type(xx)
xx

array([ 6.,  7.,  9.,  5.,  8., 11., 10.,  4.,  3., nan,  2.,  1.])

In [5]:
# sum(dfTrainRaw["PossessionTeam"] == dfTrainRaw["FieldPosition"]) + sum(dfTrainRaw["PossessionTeam"] != dfTrainRaw["FieldPosition"])

dfTrainRaw.shape

(509762, 49)

### Feature Description:

- GameId - a unique game identifier
- PlayId - a unique play identifier
- Team - home or away
- X - player position along the long axis of the field. See figure below.
- Y - player position along the short axis of the field. See figure below.
- S - speed in yards/second
- A - acceleration in yards/second^2
- Dis - distance traveled from prior time point, in yards
- Orientation - orientation of player (deg)
- Dir - angle of player motion (deg)
- NflId - a unique identifier of the player
- Season - year of the season
- YardLine - the yard line of the line of scrimmage
- Quarter - game quarter (1-5, 5 == overtime)
- GameClock - time on the game clock
- PossessionTeam - team with possession
- Down - the down (1-2-3-4)
- Distance - yards needed for a first down
- FieldPosition - which side of the field the play is happening on
- HomeScoreBeforePlay - home team score before play started
- VisitorScoreBeforePlay - visitor team score before play started
- NflIdRusher - the NflId of the rushing player (importante!)
- OffenseFormation - offense formation
- OffensePersonnel - offensive team positional grouping
- DefendersInTheBox - number of defenders lined up near the line of scrimmage, spanning the width of the offensive line
- DefensePersonnel - defensive team positional grouping
- PlayDirection - direction the play is headed (left/right)
- TimeHandoff - UTC time of the handoff
- TimeSnap - UTC time of the snap
- Yards - the yardage gained on the play (you are predicting this/TARGET VAR)
- Week - week into the season
- StadiumType - description of the stadium environment
- Turf - description of the field surface
- GameWeather - description of the game weather
- Temperature - temperature (deg F)
- Humidity - humidity
- WindSpeed - wind speed in miles/hour
- WindDirection - wind direction

In [23]:
# type(dfTrainRaw)
# equivalent to summary()
dfTrainRaw.describe()

,GameId,PlayId,X,Y,S,A,Dis,Orientation,Dir,NflId,JerseyNumber,Season,YardLine,Quarter,Down,Distance,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,DefendersInTheBox,Yards,PlayerWeight,Week,Temperature,Humidity
count,509762.00,509762.00,509762.00,509762.00,509762.00,509762.00,509762.00,509744.00,509748.00,509762.00,509762.00,509762.00,509762.00,509762.00,509762.00,509762.00,509762.00,509762.00,509762.00,509696.00,509762.00,509762.00,509762.00,461230.00,503602.00
mean,2017594430.73,20175944309348.52,60.04,26.78,2.59,1.68,0.28,180.25,179.93,2344702.90,54.19,2017.49,28.37,2.50,1.52,8.33,11.15,9.50,2419203.81,6.94,4.21,253.55,9.10,60.44,55.65
std,499709.81,4997098142.10,25.44,7.24,1.38,0.98,0.15,104.30,106.32,641887.14,28.38,0.50,13.01,1.14,0.67,3.80,10.28,9.37,535778.34,0.99,6.44,48.37,5.00,17.57,25.43
min,2017090700.00,20170907000118.00,0.66,1.85,0.00,0.00,0.00,0.00,0.00,222.00,1.00,2017.00,1.00,1.00,1.00,1.00,0.00,0.00,234.00,1.00,-14.00,153.00,1.00,9.00,0.00
25%,2017110504.00,20171105041292.00,39.09,22.73,1.50,0.94,0.17,90.63,89.83,2532865.00,26.00,2017.00,20.00,1.00,1.00,6.00,3.00,0.00,2540011.00,6.00,1.00,210.00,5.00,48.00,43.00
50%,2017123109.00,20171231090770.00,60.29,26.79,2.47,1.54,0.27,179.70,180.09,2543765.00,56.00,2017.00,29.00,3.00,1.00,10.00,9.00,7.00,2552582.00,7.00,3.00,245.00,9.00,62.00,59.00
75%,2018110403.00,20181104033931.00,80.97,30.86,3.56,2.29,0.39,270.40,270.19,2555343.00,78.00,2018.00,39.00,4.00,2.00,10.00,17.00,14.00,2557976.00,8.00,6.00,305.00,14.00,73.00,71.00
max,2018123015.00,20181230154157.00,116.84,56.45,9.41,14.71,1.39,360.00,360.00,2561671.00,99.00,2018.00,50.00,5.00,4.00,40.00,57.00,51.00,2561468.00,11.00,99.00,380.00,17.00,97.00,100.00


In [9]:
# sort by one column
# dfTrainRaw.sort_values(by = 'Distance').tail()
# select rows and/or columns
# dfTrainRaw.loc[8:11, ['abv', 'ash', 'color']]
# group_by
# dfTrainRaw.groupby('Quarter').mean()
dfTrainRaw.groupby('Quarter').agg(['mean', 'count', 'std'])

GameId                            PlayId                        \
               mean   count       std            mean   count           std   
Quarter                                                                       
1        2017595762  132066 499873.96  20175957624556  132066 4998739603.99   
2        2017592786  122430 499775.44  20175927863872  122430 4997754452.48   
3        2017595411  125026 499616.87  20175954116759  125026 4996168681.49   
4        2017592041  126434 499556.24  20175920416688  126434 4995562471.83   
5        2017648282    3806 497038.47  20176482826330    3806 4970384837.79   

            X                   Y                 S                 A          \
         mean   count   std  mean   count  std mean   count  std mean   count   
Quarter                                                                         
1       59.22  132066 24.76 26.72  132066 7.18 2.62  132066 1.40 1.70  132066   
2       60.65  122430 26.64 26.75  122430 7.30 2.55  122430 1.38 1.69  122430   
3       60.19  125026 24.85 26.79  125026 7.28 2.61  125026 1.38 1.68  125026   
4       60.06  126434 25.53 26.86  126434 7.20 2.58  126434 1.37 1.65  126434   
5       63.43    3806 24.01 26.92    3806 7.26 2.52    3806 1.33 1.64    3806   

              Dis              Orientation                   Dir          \
         std mean   count  std        mean   count    std   mean   count   
Quarter                                                                    
1       0.99 0.28  132066 0.15      180.39  132056 104.37 180.19  132059   
2       0.98 0.28  122430 0.15      179.79  122427 104.53 179.49  122427   
3       0.97 0.28  125026 0.14      180.73  125025 103.98 180.15  125026   
4       0.97 0.28  126434 0.14      180.00  126430 104.31 179.92  126430   
5       0.95 0.27    3806 0.14      182.42    3806 104.04 178.66    3806   

                  NflId                   JerseyNumber                Season  \
           std     mean   count       std         mean   count   std    mean   
Quarter                                                                        
1       106.41  2337288  132066 651889.24        54.32  132066 28.47 2017.49   
2       105.81  2342590  122430 644901.93        53.87  122430 28.41 2017.48   
3       106.42  2342800  125026 644434.45        54.09  125026 28.43 2017.49   
4       106.58  2355510  126434 626740.22        54.44  126434 28.21 2017.48   
5       107.37  2373382    3806 602041.14        54.89    3806 28.20 2017.54   

                     YardLine               Down              Distance  \
          count  std     mean   count   std mean   count  std     mean   
Quarter                                                                  
1        132066 0.50    28.89  132066 12.66 1.50  132066 0.65     8.46   
2        122430 0.50    27.24  122430 13.49 1.54  122430 0.69     8.20   
3        125026 0.50    28.86  125026 12.72 1.45  125026 0.63     8.42   
4        126434 0.50    28.42  126434 13.15 1.60  126434 0.73     8.22   
5          3806 0.50    29.09    3806 11.82 1.48    3806 0.68     8.36   

                     HomeScoreBeforePlay               VisitorScoreBeforePlay  \
          count  std                mean   count   std                   mean   
Quarter                                                                         
1        132066 3.65                2.18  132066  3.54                   1.71   
2        122430 4.16                7.83  122430  6.18                   6.19   
3        125026 3.61               14.27  125026  8.53                  12.20   
4        126434 3.75               20.35  126434 10.49                  17.83   
5          3806 3.29               20.76    3806  8.67                  20.69   

                     NflIdRusher                   DefendersInTheBox          \
          count  std        mean   count       std              mean   count   
Quarter                                                                        

In [10]:
# remove some unused columns and save them in a data.frame
dfUnusedColumns = dfTrainRaw[["NflId", "DisplayName", "Season", "JerseyNumber"]]

dfFeatures = dfTrainRaw.drop(["DisplayName", "Season", "JerseyNumber"], axis = 1)

In [20]:
type(dfFeatures)

pandas.core.frame.DataFrame

In [21]:
# how many nulls for each column
dfFeatures.isnull().sum()

GameId                        0
PlayId                        0
Team                          0
X                             0
Y                             0
S                             0
A                             0
Dis                           0
Orientation                  18
Dir                          14
NflId                         0
YardLine                      0
Quarter                       0
GameClock                     0
PossessionTeam                0
Down                          0
Distance                      0
FieldPosition              6424
HomeScoreBeforePlay           0
VisitorScoreBeforePlay        0
NflIdRusher                   0
OffenseFormation            110
OffensePersonnel              0
DefendersInTheBox            66
DefensePersonnel              0
PlayDirection                 0
TimeHandoff                   0
TimeSnap                      0
Yards                         0
PlayerHeight                  0
PlayerWeight                  0
PlayerBi

In [24]:
null_columns = dfFeatures.columns[dfFeatures.isnull().any()]
null_columns
# dfFeatures[dfFeatures.isnull().any(axis = 1)][null_columns].head(15)

Index(['Orientation', 'Dir', 'FieldPosition', 'OffenseFormation',
       'DefendersInTheBox', 'StadiumType', 'GameWeather', 'Temperature',
       'Humidity', 'WindSpeed', 'WindDirection'],
      dtype='object')

In [ ]:
# select all the rows with at least 1 null element
dfFeatures[dfFeatures.isnull().any(axis = 1)].head(10)

In [37]:
# drop features
dfFeatures = dfTrainRaw.drop(["DisplayName", "Season", "JerseyNumber", "StadiumType", "GameWeather", 
                              "Temperature", "Humidity", "WindSpeed", "WindDirection"], axis = 1)
# drop rows with NA 
dfFeatures = dfFeatures.dropna()

In [38]:
dfFeatures.head(20)

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,Turf
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
5,2017090700,20170907000118,away,75.06,24.00,1.01,0.32,0.18,308.34,95.01,2543494,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-2,252,03/19/1991,Auburn,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
6,2017090700,20170907000118,away,74.11,16.64,1.11,0.83,0.02,357.23,322.59,2543637,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,5-11,190,05/17/1992,Oregon,CB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
7,2017090700,20170907000118,away,73.37,18.73,1.24,0.74,0.13,328.52,270.04,2543851,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,193,04/04/1991,Rice,CB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
8,2017090700,20170907000118,away,56.63,26.90,0.26,1.86,0.28,344.70,55.31,2550257,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-2,208,03/05/1990,Brigham Young,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
9,2017090700,20170907000118,away,73.35,38.83,4.55,0.76,0.51,75.47,190.84,2552488,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,197,01/09/1993,Washington,CB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf


In [41]:
dfRunningBack = dfFeatures[dfFeatures["NflId"] == dfFeatures["NflIdRusher"]]
dfChosen = dfRunningBack[dfRunningBack["NflId"] == 2557917]
dfChosen = dfChosen[dfChosen["GameId"] == 2017090700]
dfChosen.head(10)
#xx = dfRunningBack[dfRunningBack["FieldPosition"].isnull()]
#xx["GameId"].value_counts(ascending=False)
# dfChosen = dfChosen[dfChosen["GameId"] == 2557917]

# xx = dfChosen[["X", "Y", "FieldPosition", "HomeTeamAbbr", "VisitorTeamAbbr"]]
#xx.isnull().sum()

#dd = ['Female', 'Male', 'Non-binary', 'No Response']
# dfRunningBack[dfRunningBack["FieldPosition"].isnull()]


#yy = xx[["X", "Y", "FieldPosition"]]
#ggplot(aes(x = 'X', y = 'Y', color = 'FieldPosition'), data = yy)  + geom_point()

# fg = seaborn.FacetGrid(data = yy, hue = 'FieldPosition', aspect = 1.61)
# fg.map(pyplot.scatter, 'X', 'Y').add_legend()
# xx.plot.scatter(x = 'X', y = 'Y', c = 'FieldPosition') 

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,Turf
98,2017090700,20170907000395,away,29.99,27.12,3.90,2.53,0.44,34.27,157.92,2557917,25,1,12:08:00,KC,1,10,KC,7,0,2557917,SHOTGUN,"1 RB, 3 TE, 1 WR",7.00,"3 DL, 2 LB, 6 DB",right,2017-09-08T00:53:14.000Z,2017-09-08T00:53:13.000Z,7,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
186,2017090700,20170907000680,away,14.57,22.25,4.77,3.10,0.51,16.13,129.93,2557917,10,1,09:25:00,KC,1,10,KC,7,0,2557917,SINGLEBACK,"2 RB, 1 TE, 2 WR",7.00,"3 DL, 2 LB, 6 DB",right,2017-09-08T01:06:42.000Z,2017-09-08T01:06:41.000Z,9,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
230,2017090700,20170907000917,away,96.73,23.79,4.34,2.11,0.55,3.53,89.94,2557917,9,1,03:47:00,KC,2,9,NE,7,0,2557917,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.00,"3 DL, 2 LB, 6 DB",right,2017-09-08T01:13:32.000Z,2017-09-08T01:13:31.000Z,2,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
538,2017090700,20170907001664,away,91.50,30.70,4.64,3.01,0.48,230.16,347.43,2557917,23,2,07:22:00,KC,1,10,KC,17,7,2557917,SHOTGUN,"2 RB, 1 TE, 2 WR",7.00,"4 DL, 3 LB, 4 DB",left,2017-09-08T01:46:09.000Z,2017-09-08T01:46:07.000Z,3,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
626,2017090700,20170907001955,away,110.13,25.10,5.20,3.20,0.55,224.29,356.28,2557917,4,2,02:29:00,KC,1,14,KC,17,7,2557917,SHOTGUN,"1 RB, 1 TE, 3 WR",6.00,"4 DL, 1 LB, 6 DB",left,2017-09-08T01:59:42.000Z,2017-09-08T01:59:41.000Z,7,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
648,2017090700,20170907002430,away,89.83,29.78,4.80,2.37,0.58,204.09,309.62,2557917,25,3,15:00:00,KC,1,10,KC,17,14,2557917,SINGLEBACK,"2 RB, 1 TE, 2 WR",7.00,"3 DL, 2 LB, 6 DB",left,2017-09-08T02:29:54.000Z,2017-09-08T02:29:52.000Z,10,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
758,2017090700,20170907002961,away,100.73,27.25,4.41,0.99,0.53,198.09,322.80,2557917,14,3,07:08:00,KC,1,10,KC,17,21,2557917,SINGLEBACK,"2 RB, 1 TE, 2 WR",8.00,"4 DL, 3 LB, 4 DB",left,2017-09-08T02:53:47.000Z,2017-09-08T02:53:46.000Z,1,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
824,2017090700,20170907003261,away,103.81,30.67,1.96,3.33,0.20,188.05,280.23,2557917,11,3,04:21:00,KC,2,12,KC,24,21,2557917,SHOTGUN,"1 RB, 1 TE, 3 WR",7.00,"4 DL, 1 LB, 6 DB",left,2017-09-08T03:09:15.000Z,2017-09-08T03:09:14.000Z,1,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
912,2017090700,20170907003635,away,110.34,26.41,3.92,2.61,0.45,166.19,314.37,2557917,5,3,00:04:00,KC,1,15,KC,27,21,2557917,PISTOL,"1 RB, 2 TE, 2 WR",7.00,"4 DL, 1 LB, 6 DB",left,2017-09-08T03:22:23.000Z,2017-09-08T03:22:22.000Z,13,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
1000,2017090700,20170907004182,away,53.21,26.00,3.22,2.07,0.42,325.11,58.43,2557917,49,4,08:20:00,KC,2,1,KC,27,28,2557917,PISTOL,"1 RB, 2 TE, 2 WR",7.00,"4 DL, 2 LB, 5 DB",right,2017-09-08T03:46:41.000Z,2017-09-08T03:46:39.000Z,7,5-11,216,08/06/1995,Toledo,RB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf
